# modeling8-nli-binaryclassifier-TESTING

In [ ]:
# !pip install evaluate

In [1]:
import json
import os
import logging
import sys
import evaluate
import wandb
from copy import deepcopy
import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset
from transformers import (
    DataCollatorWithPadding,
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from pprint import pprint

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
def open_json(file):
    with open(file , 'r') as f: 
        data = json.load(f)
    return data

In [3]:
binary_train_file = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json'
# binary_dev_file = 'binary_ex_ctx100id_split_dev_1.json'


In [4]:
binary_train_data = open_json(binary_train_file)

In [5]:
pprint(binary_train_data[0])

{'ctx': {'id': '533920',
         'text': 'he was unsatisfied with the book. Ellison ultimately wrote '
                 'more than 2,000 pages of this second novel but never '
                 'finished it. Ellison died on April 16, 1994 of pancreatic '
                 'cancer and was interred in a crypt at Trinity Church '
                 'Cemetery in the Washington Heights neighborhood of Upper '
                 'Manhattan. He was survived by his second wife, Fanny Ellison '
                 '(November 27, 1911 – November 19, 2005). "Invisible Man" won '
                 'the 1953 US National Book Award for Fiction. The award was '
                 'his ticket into the American literary establishment. He '
                 'eventually was admitted to the American Academy of Arts and '
                 "Letters, received two President's",
         'title': 'Ralph Ellison'},
 'em': '0',
 'id': 1,
 'question': 'how many pages is invisible man by ralph ellison'}


In [6]:
class BinaryCustomDataset(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length):
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + \
                 ' title: ' + self.instances[idx]['ctx']['title'] + \
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [7]:
config_name = None
model_name_or_path = 'roberta-large'

In [8]:
# config = AutoConfig.from_pretrained(
#         model_args.config_name if model_args.config_name else model_args.model_name_or_path,
#         num_labels=data_args.num_labels,
# )

In [9]:
config = AutoConfig.from_pretrained(
        config_name if config_name else model_name_or_path,
        num_labels=2,
)

In [10]:
pprint((config.num_labels))

2


In [11]:
tokenizer_name = None

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_name if tokenizer_name else model_name_or_path,
)

In [13]:
pprint(tokenizer)

PreTrainedTokenizerFast(name_or_path='roberta-large', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})


In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

In [15]:
print(type(model))

<class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>


In [16]:
print(model.config.num_labels)

2


In [17]:
max_seq_length = 200

In [18]:
# train_dataset = BinaryCustomDataset(binary_train_data,
#                                       tokenizer,
#                                       max_seq_length)

In [19]:
class BinaryCustomDatasetShuffle(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def _shuffle(self, instances):
        return random.shuffle(instances)

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + \
                 ' title: ' + self.instances[idx]['ctx']['title'] + \
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [20]:
train_dataset_shuffle = BinaryCustomDatasetShuffle(binary_train_data,
                                      tokenizer,
                                      max_seq_length,
                                      shuffle = True)

In [21]:
data_collator = DataCollatorWithPadding(
        tokenizer,
        pad_to_multiple_of=8,
    )

In [22]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)

    accr_ = metric.compute(predictions=preds, references=p.label_ids)["accuracy"]
    prec_ = metric_prec.compute(predictions=preds, references=p.label_ids)["precision"]
    recall_ = metric_recall.compute(predictions=preds, references=p.label_ids)["recall"]
    f1_ = metric_f1.compute(predictions=preds, references=p.label_ids)["f1"]

    return {
        'accuracy' : accr_,
        'precision' : prec_,
        'recall' : recall_,
        'f1' : f1_,
    }

In [23]:
from torch.utils.data import DataLoader

In [24]:
# data_loader = DataLoader(
#             train_dataset,
#             batch_size=8,
#             sampler=None,
#             collate_fn=data_collator,
#             drop_last=False,
#             num_workers=2,
#             pin_memory=False
#         )

In [26]:
data_loader_s = DataLoader(
            train_dataset_shuffle,
            batch_size=64,
            sampler=None,
            collate_fn=data_collator,
            drop_last=False,
            num_workers=2,
            pin_memory=False
        )

In [27]:
cnt = 0
for i in data_loader:
    print(i['labels'])
    cnt += 1
    if cnt == 10:
        break

NameError: name 'data_loader' is not defined

In [28]:
cnt = 0
for k in data_loader_s:
    print(k['labels'])
    cnt += 1
    if cnt == 10:
        break

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0])
tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0])
tensor([1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length):
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length
    
    def __len__(self):
        return len(self.instances)
    
    def __getitem__(self, idx):
        input_ = [' ' + self.instances[idx]['question']] + self.instances[idx]['text']
        input_txt =  f' { self.sep_token } '.join(input_) + ' '
        
        output = self.tokenizer(
            input_txt, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True, 
            add_special_tokens=True, 
            max_length=self.max_length)
        
        item = {key : val for key, val in output.items()}
        item['labels'] = torch.tensor(self.instances[idx]['labels'])
        
        return item

In [ ]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)
args = ["--model_name_or_path", 'allenai/longformer-large-4096', '--output_dir', './']
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)


In [ ]:
logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )


In [ ]:
# Set seed before initializing model.
set_seed(training_args.seed)

In [ ]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=model_args.num_labels,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    )
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    config=config,
)

In [ ]:
if training_args.do_train:
    instances, cut_off, total_questions = preprocessing_data(
        data_args.train_file, 
        data_args.sample_size, 
        data_args.position)
    
    train_instance = instances[data_args.dev_size:]
    dev_instance = instances[:data_args.dev_size]
    
    train_dataset = CustomDataset(train_instance, 
                               tokenizer, 
                               model_args.max_seq_length)
    dev_dataset = CustomDataset(train_instance, 
                               tokenizer, 
                               model_args.max_seq_length)
    
    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

if training_args.do_eval:
    instances, cut_off, total_questions = preprocessing_data(
        data_args.test_file, 
        data_args.sample_size, 
        data_args.position)
    
    test_dataset = CustomDataset(instances, 
                               tokenizer, 
                               model_args.max_seq_length)
    
    


In [ ]:
# Get the metric function
metric = evaluate.load("xnli")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)


In [ ]:
 # Initialize Trainer
data_collator = DataCollatorWithPadding(
    tokenizer, 
    pad_to_multiple_of=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_train else None,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    metrics = train_result.metrics
    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.save_model()  # Saves the tokenizer too for easy upload

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    
# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate(eval_dataset=eval_dataset)

    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
metric

In [ ]:
data_args.dataset_name = a
    

In [ ]:
data.max_seq_length

In [ ]:
training_args.fp16

In [ ]:
bb

In [ ]:
def main():
    parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )
    
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    
    